In [1]:
from ISLP import load_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = load_data('Default')

### 1a)

In [3]:
data

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# select variables
y = data[['default']]
x = data[['balance', 'income']]

LE = LabelEncoder()
y = LE.fit_transform(y)

regressor = LogisticRegression(fit_intercept=True)
regressor.fit(x, y)

d:\Anaconda\envs\MDL2\Lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [5]:
print('Intercept:', regressor.intercept_)
print('Estimates/Coefficients:', regressor.coef_)

Intercept: [-11.54047811]
Estimates/Coefficients: [[5.64710797e-03 2.08091984e-05]]


### b)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut, cross_val_score

crossval = LeaveOneOut()

# split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=5)

# instantiate and fit model 1 on training set
model_1 = LogisticRegression(fit_intercept=True, random_state=42)
model_1.fit(x_train, y_train)

# evaluate on test set
error_score = cross_val_score(model_1, x_test, y_test, 
                              scoring='neg_mean_squared_error',
                              cv=crossval, n_jobs=-1)

# check RMSE
print('LOOCV test error estimate (RMSE):', np.sqrt(np.mean(np.abs(error_score))))


LOOCV test error estimate (RMSE): 0.1876166303929372


### c)

In [7]:
# Binarize default values and one hot encode (dummy variable) student variable
labelEnc = LabelEncoder()

# copy data
data_duplicate = data.copy()

# fit and transform
data_duplicate['default'] = labelEnc.fit_transform(data_duplicate['default'])
dummies = pd.get_dummies(data_duplicate['student'], prefix='student')
data_duplicate = pd.concat([data_duplicate.drop(columns='student'), dummies], axis=1)

data_duplicate


,default,balance,income,student_No,student_Yes
0,0,729.526495,44361.625074,1,0
1,0,817.180407,12106.134700,0,1
2,0,1073.549164,31767.138947,1,0
3,0,529.250605,35704.493935,1,0
4,0,785.655883,38463.495879,1,0
...,...,...,...,...,...
9995,0,711.555020,52992.378914,1,0
9996,0,757.962918,19660.721768,1,0
9997,0,845.411989,58636.156984,1,0
9998,0,1569.009053,36669.112365,1,0


In [8]:
x1 = data_duplicate[['income', 'balance', 'student_No', 'student_Yes']]
y1 = data_duplicate[['default']]

crossval = LeaveOneOut()

# split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1,test_size=0.25, random_state=5)

# instantiate and fit model 1 on training set
model_2 = LogisticRegression(fit_intercept=True, random_state=42)
model_2.fit(x1_train, y1_train)

# evaluate on test set
error_score = cross_val_score(model_2, x1_test, y1_test, 
                              scoring='neg_mean_squared_error',
                              cv=crossval, n_jobs=-1)

# check RMSE
print('LOOCV test error estimate (RMSE):', np.sqrt(np.mean(np.abs(error_score))))

d:\Anaconda\envs\MDL2\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LOOCV test error estimate (RMSE): 0.1865475810617763


Given the RMSE scores:

RMSE Model 1 = 0.1876166303929372

RMSE Model 2 = 0.1865475810617763

Adding a dummy isn't neccessary; given the random selection of data (for reproducibiity) - the difference between the two models is barely noticeable.

### 2a)

In [23]:
def boot_estimates(data, ind):
    samples = data.iloc[ind]
    X = data_duplicate[['income', 'balance', 'student_No', 'student_Yes']]
    Y = data_duplicate.default
    mod = LogisticRegression(fit_intercept=True, random_state=42)
    mod.fit(X, Y)
    intcpt = mod.intercept_[0]
    cofs = mod.coef_[0]
    #results = pd.DataFrame({'Intercept': intcpt, 'Coefficients': cofs})
    return intcpt, cofs

### b)

In [24]:
results = np.empty((1000, 4))
for i in range(1000):
    ind = np.random.choice(data_duplicate.index, size=len(data_duplicate), replace=True)
    _ ,results[i,:] = boot_estimates(data_duplicate, ind)

In [26]:
std_errors = np.std(results, axis=0)

In [27]:
std_errors

array([2.60208521e-18, 6.85215773e-17, 1.53210777e-14, 7.19424520e-14])

Errors are displayed in the following order:

Income -> Balance -> Student_No -> Student_Yes